In [1]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime

In [2]:
test_message = "Rotator Night Observing integration test"

The next cell opens the database for query. 

In [3]:
con = sqlite3.connect(os.path.expanduser("~/develop/one_filt_v1.4_0yrs_1.db"))

Once the database is opened/connected we can query for all the entries in the `SummaryAllProps` database, which contains the information about each target observed. 

In [4]:
df = pd.read_sql_query("SELECT * from SummaryAllProps", con)

In [5]:
d = salobj.Domain()

In [6]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")
mtm = salobj.Remote(d, "MTMount")

In [14]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task,
                     mtm.start_task)

[None, None, None, None]

RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages


In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=10.)
await mtm.cmd_start.start(timeout=10.)
await mtm.cmd_enable.start(timeout=10.)

Start the loop on the different field positions df given by the scheduler

In [8]:
### Read the scheduler output positions for the next 8 hours
print("Number of visits:", len(df))

Number of visits: 971


In [9]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

# Move to 0 deg Starting Position

In [22]:
now = datetime.now()
print("Start from Park position", now)

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)  

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)

RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages


Start from Park position 2019-12-12 18:31:35.603497


RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 30 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages


Time error=-56.99 sec
0d00m00s
Waiting 30s


In [23]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)
await asyncio.sleep(0.5)

# Start Test

In [24]:
now = datetime.now()
print("Start Test", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message}")

Start Test 2019-12-12 18:32:27.874176


False

In [25]:
## start the loop that take a sky position one at the time and point and track

#for field in range(len(df)): 
for field in range(3): 
    print(field)
    target_name = df["note"][field]
    alt = df['altitude'][field] * u.deg
    az = df['azimuth'][field] * u.deg
    rot_tel = Angle(df["rotTelPos"][field]*u.deg)
    para_ang = Angle(df["paraAngle"][field]*u.deg)
    rot_pa = para_ang-180*u.deg+rot_tel.wrap_at('180d')
    visit_time = df['visitTime'][field]  # how long the visit lasted. Usefull so you can wait between visits
    print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}")

#Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.
    time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
    curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")

# Compute RA/Dec for commanded az/el
    cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
    cmd_radec = cmd_elaz.transform_to(ICRS) # This is the RA/Dec of the target

    print(f"RA: {cmd_radec.ra}\nDec: {cmd_radec.dec}\nRotPA: {rot_pa}")
    
    await rot.cmd_trackStart.start(timeout=30.)
    
    ack = await mtptg.cmd_raDecTarget.set_start(
       targetName=target_name,
       targetInstance=ATPtg.TargetInstances.CURRENT,
       frame=ATPtg.CoordFrame.ICRS,
        epoch=2000,  # should be ignored: no parallax or proper motion
        equinox=2000,  # should be ignored for ICRS
        ra=cmd_radec.ra.hour,
        declination=cmd_radec.dec.deg,
        parallax=0,
        pmRA=0,
        pmDec=0,
        rv=0,
        dRA=0,
        dDec=0,
        rotPA=rot_pa.deg,
        rotFrame=ATPtg.RotFrame.TARGET,
        rotMode=ATPtg.RotMode.FIELD,
        timeout=10.
    )

    print("Waiting", visit_time, "s")
    await asyncio.sleep(visit_time)
    
    #Workaround for Rotator failing between targets
    await mtptg.cmd_stopTracking.start(timeout=10.)
    await mtm.cmd_stop.start(timeout=10.)
    await rot.cmd_stop.start(timeout=10.)
    await asyncio.sleep(0.5)
          
now = datetime.now()
print("This is the end of the observing run", now)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 81 messages


0
Alt: 73.57495921073964 deg
Az: 4.273461703999071 deg
RotTel: 25.191637478972904 deg


RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 82 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 83 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 17 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 83 messages


RA: 289.285339575777 deg
Dec: -13.896206862587672 deg
RotPA: 28.993384547298348 deg
Waiting 31.0 s
1
Alt: 68.83074605748851 deg
Az: 8.315007012391357 deg
RotTel: 25.832374501812186 deg
RA: 291.30618345598793 deg
Dec: -9.306030608975567 deg
RotPA: 33.10452967576464 deg
Waiting 31.0 s
2
Alt: 66.59882514695401 deg
Az: 13.502094205548591 deg
RotTel: 25.87354311047927 deg
RA: 293.79417544706337 deg
Dec: -7.4357595341524405 deg
RotPA: 37.60883293952574 deg


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1952084016, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Low-level controller in substate 2.0 instead of <EnabledSubstate.STATIONARY: 0>')

# Move to Park position 

In [ ]:
now = datetime.now()
print("Move to park position", now)

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 1s")
await asyncio.sleep(1.)

In [ ]:
now = datetime.now()
print("Stopping tracking", now)

#we need to stop the pointing commands before stopping the rotator. This will be integrated in the pointing code
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Test Complete

In [ ]:
now = datetime.now()
print("Test Complete",now)

script.evt_logMessage.set_put(level=logging.INFO+1,
                          message=f"END - {test_message}")

# Additional Error Handling Commands

# Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=30)

Wait for override off

In [ ]:
await rot.cmd_clearError.start(timeout=30)

In [17]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 42 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 43 messages


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [21]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteEvent(MTMount, 0, mountState) falling behind; read 24 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 40 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages


# Move to sync position with CCW

In [ ]:
print("Move to X.X deg starting position")

await rot.cmd_positionSet.set_start(angle=19.04,timeout=30.)

await rot.cmd_move.start(timeout=30.)

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [16]:
await rot.cmd_enterControl.start(timeout=10.)

RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 89 messages
RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 44 messages


In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await mtm.cmd_clearerror.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

Bring CCW Down

In [20]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Application) falling behind; read 100 messages
RemoteTelemetry(Rotator, 0, Motors) falling behind; read 100 messages
RemoteEvent(MTMount, 0, mountState) falling behind; read 11 messages
RemoteTelemetry(Rotator, 0, Electrical) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 11 messages


Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

In [26]:
await mtptg.cmd_stopTracking.start(timeout=30)

RemoteTelemetry(MTMount, 0, Camera_Cable_Wrap) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteEvent(Rotator, 0, heartbeat) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, skyEnvironment) falling behind; read 36 messages
RemoteTelemetry(MTPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(MTPtg, 0, currentTimesToLimits) falling behind; read 36 messages
RemoteEvent(MTPtg, 0, weatherDataApplied) falling behind; read 18 messages


In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)